In [1]:
import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    folder_name = 'CSCI544-Project'
    folder_path = f'/content/drive/MyDrive/{folder_name}'
    os.chdir(folder_path)
    print("Current working directory is: " + os.getcwd())
    print(os.listdir())
except:
    os.chdir("../")
finally:
    print("Current working directory is: " + os.getcwd())
    print(os.listdir())

Current working directory is: /Users/swapnildhanwal/Documents/USC/CSCI 544/Project/csci-544-project
['embeddings', '.DS_Store', 'defect', 'sample', '.git', 'data', '.idea', 'src']


In [3]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [4]:
BASE_EMBEDDINGS_PATH = "./t5p_small_embeddings/"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
def load_tensors(file_name):
    X = []
    y = []
    with open(BASE_EMBEDDINGS_PATH + file_name) as f:
        line = f.readline()
        while True:
            if not line:
                break
            data = json.loads(line)
            X.append(data["embeddings"])
            y.append(data["label"])
            line = f.readline()
    X = np.array(X)
    y = np.array(y)
    X = torch.tensor(X, dtype=torch.float32).clone().detach().to(DEVICE).reshape(-1, 256)
    y = torch.tensor(y, dtype=torch.float32).clone().detach().to(DEVICE).reshape(-1, 1)
    return X, y

def get_model():
    model = nn.Sequential(
        nn.Linear(256, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid()
    ).to(DEVICE)
    loss_fn = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    return model, loss_fn, optimizer

def train(model, loss_fn, optimizer, X: torch.tensor, y: torch.tensor, n_epochs: int, batch_size: int):
    for epoch in range(n_epochs):
        for i in range(0, len(X), batch_size):
            Xbatch = X[i:i+batch_size]
            y_pred = model(Xbatch)
            ybatch = y[i:i+batch_size]
            loss = loss_fn(y_pred, ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Finished epoch {epoch}, latest loss {loss}')

def f1_score(y_true, y_pred, threshold=0.5, eps=1e-8):
    y_pred_binarized = (y_pred > threshold).float()
    tp = (y_true * y_pred_binarized).sum(dim=0)
    fp = ((1 - y_true) * y_pred_binarized).sum(dim=0)
    fn = (y_true * (1 - y_pred_binarized)).sum(dim=0)
    precision = tp / (tp + fp + eps)
    recall = tp / (tp + fn + eps)
    f1 = 2 * (precision * recall) / (precision + recall + eps)
    return f1

def evaluate(model: nn.Sequential, X, y):
    with torch.no_grad():
        y_pred = model(X)
    accuracy = (y_pred.round() == y).float().mean()
    print(f"Accuracy {accuracy}")
    print(f1_score(y, y_pred))

In [6]:
X, y = load_tensors("train_0.jsonl")
print(X.shape)
print(y.shape)

FileNotFoundError: [Errno 2] No such file or directory: './t5p_small_embeddings/train_0.jsonl'

In [7]:
import os
print(os.getcwd())

/Users/swapnildhanwal/Documents/USC/CSCI 544/Project/csci-544-project/src
